## Pulsed Measurement bit

### Import dependencies and optimize function

In [9]:
odmr = odmrlogic
import time
from tqdm import tqdm
import glob
from pandas import read_csv
import os
poi_m = poimanagerlogic
pml = pulsedmeasurementlogic
def optimize(poi):
    if not poi == 'ref_auto_poi':
        #poi_m.go_to_poi(poi)
        confocal.refocus_clicked()
        time.sleep(1)
        while optimizerlogic.module_state()=='locked':
           time.sleep(1)
        return
    poi_m.optimise_poi_position(name=poi)
    while optimizerlogic.module_state()=='locked':
        time.sleep(1)    

end_time =30*60
save_time =30*60
opt_time = 10*60

# end_time = 60*60
# save_time = 30*60
# opt_time = 10*60
my_range = np.arange(8e6, 10e6)

### Run any GUI loaded regular pulsed measurement and save with tag 

#### optimize - continue_pulsed - wait - pause_pulsed - optimize - save - repeat

In [10]:
poi = 'site_3_10_4'
sequence_name = 'rabi_AWG'
fit = 'sineexp'
frequency_power_MW = '2772558KHzs_0p5amp'
#optimize('ref_auto_poi ')
optimize(poi)
pulsedmeasurementlogic.continue_pulsed_measurement()
time_end = time.monotonic()
while (time.monotonic() - time_end) < end_time:
    os.system('cmd /c "shutdown -a"')
    time60 = time.monotonic()
    while (time.monotonic() - time60) < save_time:
        time10 = time.monotonic()
        while (time.monotonic() - time10) < opt_time:
            pulsedmasterlogic.do_fit(fit, False)
            time.sleep(10)
        pulsedmeasurementlogic.pause_pulsed_measurement()
        time.sleep(5)
        #optimize('ref_auto_poi')
        optimize(poi)
        pulsedmeasurementlogic.continue_pulsed_measurement()
    pml.save_measurement_data(tag=f'{sequence_name}_{poi}_{frequency_power_MW}')
pulsedmeasurementlogic.pause_pulsed_measurement()
pml.save_measurement_data(tag=f'{sequence_name}_{poi}_{frequency_power_MW}_final')

'C:/Data\\2021\\01\\20210127\\PulsedMeasurement'

In [11]:
# poi = 'site_3_10_4'
# sequence_name = 'pi_pulse_test_AWG_22p16ns'
# frequency_power_MW = '2772138KHz_0p5amp'
pml.save_measurement_data(tag=f'{sequence_name}_{poi}_{frequency_power_MW}_final')

'C:/Data\\2021\\01\\20210127\\PulsedMeasurement'

In [4]:
for i in range(5):
    time.sleep(5)
    optimize(poi)

### Run ODMR

#### optimize - odmr - save 
For long ODMR on a single spot

In [10]:
poi = 'site_1_11_14'
optimize(poi)
odmr.continue_odmr_scan()
time_end = time.monotonic()
while (time.monotonic() - time_end) < end_time:
    time60 = time.monotonic()
    while (time.monotonic() - time60) < save_time:
        time10 = time.monotonic()
        while (time.monotonic() - time10) < opt_time:
            time.sleep(10)
        odmr.stop_odmr_scan()
        time.sleep(5)
        optimize(poi)
        odmr.continue_odmr_scan()
    odmr.save_odmr_data(tag=f'{poi}_podmr_MW1_m24dbm_pi318ns_200uW')
odmr.stop_odmr_scan()    
odmr.save_odmr_data(tag=f'{poi}_podmr_MW1_m24dbm_pi318ns_200uW_final')

# Run pulsed measurements after setting parameters
#### poi from loop - optimize - take one set of params - set_params - continue_pulsed - wait - pause_pulsed - optimize - save - stop - repeat

In [14]:
def run_measurement(set_params, measurement_tag, fit=False):
    for poi in poi_parameters.keys():
        #optimize('ref_auto_poi')
        optimize(poi)
        os.system('cmd /c "shutdown -a"')
        for i in range(len(poi_parameters[poi])):
            params = poi_parameters[poi][i]
            set_params(params)
            time.sleep(3)
            pulsedmeasurementlogic.start_pulsed_measurement()
            time_end = time.monotonic()
            while (time.monotonic() - time_end) < end_time:
                time60 = time.monotonic()
                while (time.monotonic() - time60) < save_time:
                    time10 = time.monotonic()
                    time.sleep(1) #optimiser is running before and after saving. Perhaps needs a small delay here?
                    while (time.monotonic() - time10) < opt_time:
                        time.sleep(10)
                    pulsedmeasurementlogic.pause_pulsed_measurement()
                    time.sleep(5)
                    #optimize('ref_auto_poi')
                    optimize(poi)
                    pulsedmeasurementlogic.continue_pulsed_measurement()
                if fit:
                    pulsedmasterlogic.do_fit('rabi')
                    time.sleep(0.5)
                pulsedmeasurementlogic.pause_pulsed_measurement()
                time.sleep(5)
                pml.save_measurement_data(tag=f'{measurement_tag}_{poi}_params_{params}')
                time.sleep(2)
                pulsedmasterlogic.do_fit('No Fit')
                pulsedmeasurementlogic.continue_pulsed_measurement()
            pulsedmeasurementlogic.pause_pulsed_measurement()  #CHANGED TO PAUSE
            if fit:
                pulsedmasterlogic.do_fit('rabi')
                time.sleep(0.5)
            pml.save_measurement_data(tag=f'{measurement_tag}_{poi}_params_{params}')
            if fit:
                rabi_frequency = pulsedmasterlogic.fit_container.current_fit_result.best_values['frequency']
                poi_parameters[poi][i][0], ideal = determine_ideal_mw_power(rabi_frequency, poi_parameters[poi][i][0])
                if ideal:
                    print('Ideal: ', poi, poi_parameters[poi][i] )
                    del poi_parameters[poi][i]
                    if not poi_parameters[poi]:
                        del poi_parameters[poi]

### Set parameters function for T2

In [15]:
poi_parameters = {
 #   'auto_spot_2_140' : [(116e-9, -4, 2828e6, 'd_ch3'),(130e-9, -1, 2800e6, 'd_ch3')],
#      'auto_spot_2_140' : [(130e-9, -1, 2800e6, 'd_ch3')],
#      'auto_spot11': [(140e-9, -10, 2805e6, 'd_ch3'), (76e-9, -4, 2828e6,'d_ch4')],
# #     'auto_spot_2_145' : [(102e-9, -4, 2800e6), (104e-9, -6, 2827e6)],
#  #    'auto_spot25' : [(104e-9, -9, 2804e6),(97e-9, -10, 2828e6)]
#     'auto_spot53': [(128e-9, -7, 2805e6, 'd_ch3'), (104e-9, -4, 2828e6,'d_ch4')],
#     'auto_spot56': [(128e-9, -4, 2800e6, 'd_ch3'), (96e-9, -3, 2829e6,'d_ch4')],
#     'auto_spot81': [(120e-9, -3, 2800e6, 'd_ch3'), (80e-9, -1, 2830e6,'d_ch4')]
#     'auto_spot_2_179': [(125e-9, -1, 2800e6, 'd_ch3'), (120e-9, 0, 2826e6,'d_ch4')],
#     'auto_spot_2_165': [(105e-9, -2, 2805e6, 'd_ch3'), (115e-9, -2, 2829e6,'d_ch4')],
#     'auto_spot_2_106': [(95e-9, -2, 2806e6, 'd_ch3'), (100e-9, -2, 2833e6,'d_ch4')],
#     'auto_spot_2_84': [(120e-9, -2, 2801e6, 'd_ch3'), (100e-9, -2, 2833e6,'d_ch4')]
#    'ref_auto_poi': [(130e-9, -6, 2806e6, 'd_ch3')]
 #   'auto_spot_2_284': [(120e-9, -10, 2806e6, 'd_ch3'), (112e-9, -10, 2830e6,'d_ch4')]
    'auto_spot_2_284': [(100e-9, -7, 2917e6, 'd_ch4'), (120e-9, -10, 2833e6,'d_ch4')]
}

def set_T2_params(params):
    rabi_period, dbm, mw_freq, mw_ch = params
    sequencegeneratorlogic.set_generation_parameters({'laser_channel':'d_ch1',
                                                     'sync_channel': '',
                                                    'gate_channel': 'd_ch2',
                                                    'microwave_channel': mw_ch,
                                                    'microwave_frequency': 2829000000.0,
                                                    'microwave_amplitude': 0.0,
                                                    'rabi_period': rabi_period,
                                                    'laser_length': 3.2000000000000003e-06,
                                                    'laser_delay': 5e-07,
                                                    'wait_time': 1e-05,
                                                    'analog_trigger_voltage': 0.0
                                                    })
    if mw_ch == 'd_ch3':
        microwave_2.off()
        time.sleep(0.5)
        pulsedmasterlogic.set_ext_microwave_settings(power=dbm, frequency=mw_freq, use_ext_microwave=True)
        time.sleep(0.5)
    else:
        pulsedmasterlogic.set_ext_microwave_settings(use_ext_microwave=False)
        time.sleep(0.5)
        microwave_2.set_cw(mw_freq, dbm)
        time.sleep(0.5)
        microwave_2.cw_on()
        time.sleep(0.5)
    #sequencegeneratorlogic.sample_pulse_block_ensemble('hahn_echo')
    #sequencegeneratorlogic.load_ensemble('hahn_echo')
    generate_params = pulsedmasterlogic.generate_method_params['hahnecho']
    generate_params = {'name': 'hahnecho',
                      'tau_start': 50e-9,
                      'tau_step': 500e-9,
                      'num_of_points': 130}
    # Call the actual generation method using a wrapper method and pass the parameter set
    pulsedmasterlogic.generate_predefined_sequence('hahnecho', generate_params)
    # Give it a moment to generate
    time.sleep(0.5)
    # The "with_load" flag can be used to also load the waveform into the channels after creating it
    pulsedmasterlogic.sample_ensemble('hahnecho', with_load=True)
    # Wait for the sampling to finish. There is a status dictionary that can be used to check for process status.
    while pulsedmasterlogic.status_dict['sampload_busy']:
        time.sleep(0.1)
    pulsedmasterlogic.set_alternative_data_type('Delta')
    time.sleep(0.5)

### Set parameters for Rabi

In [8]:
poi_parameters = {}
pois = [
    'auto_spot11',
    'auto_spot18',
    'auto_spot25',
    'auto_spot53',
    'auto_spot56',
    'auto_spot60',
    'auto_spot81',
    'auto_spot85',
    'auto_spot_2_3',
    'auto_spot_2_9',
    'auto_spot_2_22',
    'auto_spot_2_24',
    'auto_spot_2_37',
    'auto_spot_2_58',
    'auto_spot_2_63',
    'auto_spot_2_84',
    'auto_spot_2_106',
    'auto_spot_2_117',
    'auto_spot_2_120',
    'auto_spot_2_127',
    'auto_spot_2_129',
    'auto_spot_2_139',
    'auto_spot_2_148',
    'auto_spot_2_165',
    'auto_spot_2_175',
    'auto_spot_2_179',
    'auto_spot_2_277',
    'auto_spot_2_278'
]
def get_center_frequencies(poi):
    loc = glob.glob('C:/Users/PI3/Desktop/NV Pair Lab data/Auto Rabi/*'+poi+'_*.dat')[0]
    data = read_csv(loc, skiprows=31, header=None, delimiter='\t', names=['x','y'])
    x = np.asarray(data['x'])
    y = np.asarray(data['y'])
    split = list(np.where(np.abs(x-2.87e9)<=100e4))[0][-1]
    x = x[:split]
    y = y[:split]
    result = fitlogic.make_lorentziandouble_fit(x_axis=x, data=y, estimator=fitlogic.estimate_lorentziandouble_dip)
    l0, l1 = result.params['l0_center'].value, result.params['l1_center'].value
    poi_parameters[poi] = [(-4, l0, 'd_ch3', 10e-9, 10e-9, 50),(-4, l1, 'd_ch4', 10e-9, 10e-9, 50)]

In [22]:
def determine_ideal_mw_power(rabi_frequency, p0):
    v1 = np.average(my_range)
    if int(rabi_frequency) in my_range:
        return p0, True
    p1 = p0 - 20*np.log10(rabi_frequency/v1) 
    return round(p1,2), False #Smiq only takes a power value rounded to decimals anyway

In [23]:
determine_ideal_mw_power(10e6, -4)

(-4.92, False)

In [4]:
def set_rabi_params(params):
    dbm, mw_freq, mw_ch, tau_start, tau_step, number_of_taus = params
    pulsedmasterlogic.set_generation_parameters(
                                                {'laser_channel':'d_ch1',
                                                 'sync_channel': '',
                                                'gate_channel': 'd_ch2',
                                                'microwave_channel': mw_ch,
                                                'microwave_frequency': mw_freq,
                                                'microwave_amplitude': dbm,
                                                'rabi_period': 100e-9,
                                                'laser_length': 3.2000000000000003e-06,
                                                'laser_delay': 5e-07,
                                                'wait_time': 1e-05,
                                                'analog_trigger_voltage': 0.0
                                                })
    generate_params = pulsedmasterlogic.generate_method_params['rabi']
    generate_params = {'name': 'my_rabi',
                      'tau_start': tau_start,
                      'tau_step': tau_step,
                      'num_of_points': number_of_taus}
    # Call the actual generation method using a wrapper method and pass the parameter set
    pulsedmasterlogic.generate_predefined_sequence('rabi', generate_params)
    # Give it a moment to generate
    time.sleep(0.5)
    # The "with_load" flag can be used to also load the waveform into the channels after creating it
    pulsedmasterlogic.sample_ensemble('my_rabi', with_load=True)
    # Wait for the sampling to finish. There is a status dictionary that can be used to check for process status.
    while pulsedmasterlogic.status_dict['sampload_busy']:
        time.sleep(0.1)
        
    pulsedmasterlogic.set_alternative_data_type('FFT')
    time.sleep(0.5)
    
    if mw_ch == 'd_ch3':
        pulsedmasterlogic.set_ext_microwave_settings(power=dbm, frequency=mw_freq, use_ext_microwave=True)
        time.sleep(0.5)
    else:
        pulsedmasterlogic.set_ext_microwave_settings(use_ext_microwave=False)
        microwave_2.set_cw(mw_freq, dbm)
        time.sleep(0.5)
        microwave_2.cw_on()
        time.sleep(0.5) 

In [9]:
for poi in pois[:3]:
    get_center_frequencies(poi)

In [11]:
while poi_parameters:
    run_measurement(set_rabi_params, 'Rabi_long', True)

currently loaded asset: ('my', 'PulseBlockEnsemble')
currently loaded asset: ('my', 'PulseBlockEnsemble')
currently loaded asset: ('my', 'PulseBlockEnsemble')
currently loaded asset: ('my', 'PulseBlockEnsemble')
currently loaded asset: ('my', 'PulseBlockEnsemble')
currently loaded asset: ('my', 'PulseBlockEnsemble')


In [16]:
run_measurement(set_T2_params, 'T2_long')

In [5]:
#pulsedmasterlogic.set_ext_microwave_settings(power=-1, frequency=2.8e9, use_ext_microwave=True)
sequencegeneratorlogic.sample_pulse_block_ensemble('rabi')
sequencegeneratorlogic.load_ensemble('rabi')

0

In [11]:
set_T2_params((150e-9, -1, 2800e6, 'd_ch3'))

## Set parameters for Coupled measurement

In [15]:
import pickle
import os
poi_parameters = {
    #'site_2_11_19' : [((61.5e-9, 24.5e-9, 101.5e-9, 6, 2833e6, 'd_ch4', 61e-9, 27.55e-6, 6, 2774e6, 'd_ch3', False))]
    'site_3_10_4' : [((18e-9, 7e-9, 30e-9, -3, 2772e6, 'd_ch3', 57e-9, 24e-6, 0, 2848e6, 'd_ch4', True))]
    #'site_1_11_14' : [((70e-9, 31.5e-9, 113.5e-9, 6, 2758e6, 'd_ch3', 96e-9, 1e-6, 6, 2810e6, 'd_ch4', True))]
    
}
def set_hahn_echo_coupled_params(params):
    nv_a_pi, nv_a_pi_half, nv_a_pi_three_half, nv_a_dbm, nv_a_mw_freq, nv_a_mw_ch, nv_b_pi, tau, nv_b_dbm, nv_b_mw_freq, nv_b_mw_ch, nv_a_equal_nv_1 = params
    
    blockname = 'hahn_echo_coupled'
    file = open(sequencegeneratorlogic._assets_storage_dir+'\\'+blockname+'.block', 'rb')
    data = pickle.load(file)
    data, tau_step = change_hahn_echo_coupled_data(data, nv_a_pi, nv_a_pi_half, nv_a_pi_three_half, nv_b_pi, tau, nv_a_equal_nv_1)

    filename = '{0}.block'.format(blockname)
    with open(os.path.join(sequencegeneratorlogic._assets_storage_dir, filename), 'wb') as file:
        pickle.dump(data, file)
    sequencegeneratorlogic._update_blocks_from_file()
    sequencegeneratorlogic._update_ensembles_from_file()
    sequencegeneratorlogic.sample_pulse_block_ensemble(blockname)
    sequencegeneratorlogic.load_ensemble(blockname)
    time.sleep(2)
    
    pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                           controlled_variable=10e-9 + np.arange(50) * tau_step,
                                           number_of_lasers=100, 
                                           laser_ignore_list=[], 
                                           alternating=True, 
                                           units=('s', 'arb. u.'))
    time.sleep(0.5)
    pulsedmasterlogic.set_alternative_data_type('Delta')
    time.sleep(0.5)
    
    pulsedmasterlogic.set_ext_microwave_settings(use_ext_microwave=False)
    time.sleep(0.5)
    if nv_a_mw_ch == 'd_ch3':
        microwave_1.set_cw(nv_a_mw_freq, nv_a_dbm)
        microwave_2.set_cw(nv_b_mw_freq, nv_b_dbm)
    else:
        microwave_1.set_cw(nv_b_mw_freq, nv_b_dbm)
        microwave_2.set_cw(nv_a_mw_freq, nv_a_dbm)
    time.sleep(0.5)
    microwave_1.cw_on()
    microwave_2.cw_on()
    time.sleep(0.5) 

In [14]:
blockname = 'rabi'
file = open(sequencegeneratorlogic._assets_storage_dir+'\\'+blockname+'.block', 'rb')
data = pickle.load(file)
data

PulseBlock(name='rabi', element_list=[PulseBlockElement(init_length_s=2e-08, increment_s=0.0, laser_on=False, pulse_function={}, digital_high={'d_ch1': False, 'd_ch2': False, 'd_ch3': False, 'd_ch4': False, 'd_ch5': True, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}), PulseBlockElement(init_length_s=1e-08, increment_s=1e-08, laser_on=False, pulse_function={}, digital_high={'d_ch1': False, 'd_ch2': False, 'd_ch3': True, 'd_ch4': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}), PulseBlockElement(init_length_s=2.2e-05, increment_s=0, laser_on=False, pulse_function={}, digital_high={'d_ch1': False, 'd_ch2': False, 'd_ch3': False, 'd_ch4': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': False, 'd_ch8': False}), PulseBlockElement(init_length_s=3e-06, increment_s=0, laser_on=True, pulse_function={}, digital_high={'d_ch1': True, 'd_ch2': True, 'd_ch3': False, 'd_ch4': False, 'd_ch5': False, 'd_ch6': False, 'd_ch7': True, 'd_ch8': False}), PulseBlockElement(init_lengt

### Cells to load, change, and dump pulse blocks

In [7]:
import pickle
blockname = 'hahn_echo_coupled'
file = open(sequencegeneratorlogic._assets_storage_dir+'\\'+blockname+'.block', 'rb')
data = pickle.load(file)

In [15]:
data, tau_step = change_hahn_echo_coupled_data(data, 50e-9, 25e-9, 75e-9, 80e-9, 50e-6, False)
# blockname = 'laser_on_2'
filename = '{0}.block'.format(blockname)
with open(os.path.join(sequencegeneratorlogic._assets_storage_dir, filename), 'wb') as file:
    pickle.dump(data, file)
sequencegeneratorlogic._update_blocks_from_file()
sequencegeneratorlogic._update_ensembles_from_file()

In [9]:
def change_data(data):
    data[0].init_length_s = 4.5e-06
    data[0].laser_on = False
    data[0].digital_high['d_ch1'] = False
    return data

In [174]:
sequencegeneratorlogic.sample_pulse_block_ensemble(blockname)
sequencegeneratorlogic.load_ensemble(blockname)

0

### Coupled pulse block parameters

In [16]:
def change_hahn_echo_coupled_data(data, nv_a_pi, nv_a_pi_half, nv_a_pi_three_half, nv_b_pi, tau, nv_a_equal_nv_1=True):
    tau_a = 10e-9
    tau_b = tau - tau_a - nv_b_pi
    laser_pulses = 50
    tau_step = (tau - 2*tau_a - nv_b_pi)/(laser_pulses-1)
    data[0].init_length_s = nv_a_pi_half
    data[1].init_length_s = tau  #git blame: length is spelt length, not lenght
    data[2].init_length_s = nv_a_pi
    data[3].init_length_s = tau_a
    data[3].increment_s = tau_step
    data[4].init_length_s = nv_b_pi
    data[5].init_length_s = tau_b
    data[5].increment_s = -tau_step
    data[6].init_length_s = nv_a_pi_half
    
    data[10].init_length_s = nv_a_pi_half
    data[11].init_length_s = tau
    data[12].init_length_s = nv_a_pi
    data[13].init_length_s = tau_a
    data[13].increment_s = tau_step
    data[14].init_length_s = nv_b_pi
    data[15].init_length_s = tau_b
    data[15].increment_s = -tau_step
    data[16].init_length_s = nv_a_pi_three_half
    
    if nv_a_equal_nv_1:
        data[0].digital_high['d_ch3'] = True
        data[0].digital_high['d_ch4'] = False
        data[2].digital_high['d_ch3'] = True
        data[2].digital_high['d_ch4'] = False
        data[4].digital_high['d_ch3'] = False
        data[4].digital_high['d_ch4'] = True
        data[6].digital_high['d_ch3'] = True
        data[6].digital_high['d_ch4'] = False
        data[10].digital_high['d_ch3'] = True
        data[10].digital_high['d_ch4'] = False
        data[12].digital_high['d_ch3'] = True
        data[12].digital_high['d_ch4'] = False
        data[14].digital_high['d_ch3'] = False
        data[14].digital_high['d_ch4'] = True
        data[16].digital_high['d_ch4'] = False
        data[16].digital_high['d_ch3'] = True
    else:
        data[0].digital_high['d_ch3'] = False
        data[0].digital_high['d_ch4'] = True
        data[2].digital_high['d_ch3'] = False
        data[2].digital_high['d_ch4'] = True
        data[4].digital_high['d_ch3'] = True
        data[4].digital_high['d_ch4'] = False
        data[6].digital_high['d_ch3'] = False
        data[6].digital_high['d_ch4'] = True
        data[10].digital_high['d_ch3'] = False
        data[10].digital_high['d_ch4'] = True
        data[12].digital_high['d_ch3'] = False
        data[12].digital_high['d_ch4'] = True
        data[14].digital_high['d_ch3'] = True
        data[14].digital_high['d_ch4'] = False
        data[16].digital_high['d_ch4'] = True
        data[16].digital_high['d_ch3'] = False
    
    return data, tau_step

### Run coupled measurement

In [17]:
run_measurement(set_hahn_echo_coupled_params, 'hahn_echo_coupled', fit=False)
microwave_1.off()
microwave_2.off()

0

In [5]:
for _ in range(2*60):
    pulsedmasterlogic.do_fit('sineexp', False)
    time.sleep(1)

In [9]:
poi_parameters.keys()

dict_keys(['site_3_10_4'])

In [5]:
for _ in range(10):
    optimize('site_1_11_14_35,-20')
    time.sleep(2*60)

In [ ]:
microwave_2._gpib_connection.write('')
microwave_2._command_wait('')
microwave_1._gpib_connection.query('')

digital state - on
digital source - list
digital select standard -user
digital modulation - pi/4DQPSK
digital symbol rate - ?

In [64]:
user_list = 10*' ,0, 0' + 1* ' ,0, 1' + 10*' ,0, 0' #+ ' ,1, 0'

microwave_1._gpib_connection.write(':DM:STAT ON')
microwave_1._gpib_connection.write(':DM:FORM  P4QPsk') 
#microwave_1._gpib_connection.write(':DM:FORM  8Psk')#as an example

microwave_1._gpib_connection.write(':DM:SOUR DList')

microwave_1._gpib_connection.write(':DM:DList:DATA' +user_list)

#microwave_1._gpib_connection.write(':DM:STAT ON')
time.sleep(5)
#microwave_1._gpib_connection.write(':DM:STAT OFF')

In [69]:
user_list = ' 0, 1, 0,1,1,0, 1, 0,1,1,0, 1, 0,1,1,0, 1, 0,1,1'
microwave_1._gpib_connection.write(':DM:DList:DATA' +user_list)
microwave_1._gpib_connection.query(':DM:DLISt:DATA?')

'0,1,0,1,1,0,1,0,1,1,0,1,0,1,1,0,1,0,1,1\n'

In [62]:
microwave_1._gpib_connection.query(':DM:DLISt:DATA?')

'1,1\n'

In [5]:
microwave_1._gpib_connection.write(':DM:STAT OFF')

(14, <StatusCode.success: 0>)

In [9]:
microwave_1._gpib_connection.write(':LIST:STAT ON')

(15, <StatusCode.success: 0>)

In [11]:
import pickle
import os
blockname = 'xy8_tau'
file = open(sequencegeneratorlogic._assets_storage_dir+'\\'+blockname+'.block', 'rb')
data = pickle.load(file)
for i in range(len(data[:])):
    data[i].digital_high['d_ch5'] = True
filename = '{0}.block'.format(blockname)
with open(os.path.join(sequencegeneratorlogic._assets_storage_dir, filename), 'wb') as file:
    pickle.dump(data, file)
sequencegeneratorlogic._update_blocks_from_file()
sequencegeneratorlogic._update_ensembles_from_file()
sequencegeneratorlogic.sample_pulse_block_ensemble(blockname)
sequencegeneratorlogic.load_ensemble(blockname)

0

3120
3920
4720
5520
6320
7120
7920
8720
9520
10320
11120
11920
12720
13520
14320
15120
15920
16720
17520
18320
19120
19920
20720
21520
22320
318000
3120
3920
4720
5520
6320
7120
7920
8720
9520
10320
11120
11920
12720
13520
14320
15120
15920
16720
17520
18320
19120
19920
20720
21520
22320
318000
3120
3920
4720
5520
6320
7120
7920
8720
9520
10320
11120
11920
12720
13520
14320
15120
15920
16720
17520
18320
19120
19920
20720
21520
22320
318000


Traceback (most recent call last):
  File "C:\qudi\qudi\logic\jupyterkernel\qzmqkernel.py", line 679, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-123-91f03865abb3>", line 1, in <module>
    arb_forn
NameError: name 'arb_forn' is not defined
